In [ ]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END

In [ ]:
os.environ["SERPAPI_API_KEY"] = "42a63592c032fc7a97f972c068c18c2055f7409b1886d53270e4e77d089fd71f"

llm = ChatOpenAI(
    api_key="not-needed",
    base_url="http://localhost:1234/v1",
    model="local-model",
    temperature=0,
    streaming=True
)

In [ ]:
# --- Tool Definition ---
# Instantiate the SerpAPI search tool.
search_tool = SerpAPIWrapper()

# --- State Definition ---
# This defines the "memory" or "state" that flows through the graph.
class ResearchState(TypedDict):
    topic: str
    explanation: str
    summary: str


In [ ]:
def researcher_agent(state: ResearchState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]
    
    # Use the tool to search for information. The .run() method takes the query string.
    search_results = search_tool.run(topic)
    
    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})
    
    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [ ]:
def summarizer_agent(state: ResearchState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )
    
    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})
    
    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [ ]:
# --- Graph Definition ---

workflow = StateGraph(ResearchState)

# Add the agent functions as nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the edges, which control the flow
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# Set the entry point
workflow.set_entry_point("researcher")

# Compile the graph
app = workflow.compile()

In [ ]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic for the agents to research and summarize: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)
    
    print("\n--- FINAL SUMMARY ---")
    print(final_state['summary'])